In [ ]:
from datetime import datetime

In [ ]:
import numpy as np
import pandas as pd
from eolearn.core import FeatureType
from eolearn.io import ImportFromTiff
from fs_s3fs import S3FS
from sentinelhub import SHConfig
from tqdm.auto import tqdm

In [ ]:
AWS_ACCESS_KEY_ID = ''
AWS_SECRET_ACCESS_KEY = ''
BUCKET_NAME = ''
LOC_ON_BUCKET = ''

In [ ]:
filesystem = S3FS(bucket_name=BUCKET_NAME,
                  aws_access_key_id=AWS_ACCESS_KEY_ID,
                  aws_secret_access_key=AW)

In [ ]:
config = SHConfig()
config.aws_access_key_id = AWS_ACCESS_KEY_ID
config.aws_secret_access_key = AWS_SECRET_ACCESS_KEY

In [ ]:
metadata_ms4 = pd.read_parquet(filesystem.openbin('metadata/deimos_ms4_metadata.pq'))
metadata_pan = pd.read_parquet(filesystem.openbin('metadata/deimos_pan_metadata.pq'))

In [ ]:
medians_ms4 = []
medians_pan = []
for i, sensing_time in enumerate(tqdm(filesystem.listdir(LOC_ON_BUCKET))):
    try:
        folder = f's3://{BUCKET_NAME}/{LOC_ON_BUCKET}/{sensing_time}'
        eop_ms4 = ImportFromTiff((FeatureType.DATA, 'MS4'), folder=folder, config=config).execute(
            filename=['B04.tiff', 'B03.tiff', 'B02.tiff', 'B01.tiff'])
        eop = ImportFromTiff((FeatureType.DATA, 'PAN'), folder=folder,
                             config=config).execute(eop_ms4, filename='PAN.tiff')
        eop.timestamp = [datetime.strptime(sensing_time, '%Y-%m-%d_%H-%M-%S')]

        data = eop.data['MS4']
        mask = data[..., 0] > 0
        data_masked = data[mask, :]
        median_ms4 = np.median(data_masked, axis=0)

        medians_ms4.append({'sensing_time': eop.timestamp,
                           'STX_MEDIAN_1': median_ms4[0], 'STX_MEDIAN_2': median_ms4[1],
                            'STX_MEDIAN_3': median_ms4[2], 'STX_MEDIAN_4': median_ms4[3]})

        data = eop.data['PAN']
        mask = data[..., 0] > 0
        data_masked = data[mask, :]
        median_pan = np.median(data_masked, axis=0)
        medians_pan.append({'sensing_time': eop.timestamp,
                            'STX_MEDIAN_1': median_pan[0]})

    except Exception as e:
        print(e)
        print(f'Failed to proces sensing time: {sensing_time}')

In [ ]:
ms4 = pd.DataFrame(medians_ms4)
ms4.sensing_time = ms4.sensing_time.apply(lambda x: x[0])
ms4.sensing_time = ms4.sensing_time.apply(lambda x: str(x).replace(' ', 'T'))
metadata_ms4_median = metadata_ms4.set_index('START_TIME').join(ms4.set_index('sensing_time')).reset_index()

with filesystem.openbin('metadata/deimos_ms4_metadata.pq', 'wb') as f:
    metadata_ms4_median.to_parquet(f)

In [ ]:
pan = pd.DataFrame(medians_pan)
pan.sensing_time = pan.sensing_time.apply(lambda x: x[0])
pan.sensing_time = pan.sensing_time.apply(lambda x: str(x).replace(' ', 'T'))
pan_median = metadata_pan.set_index('START_TIME').join(pan.set_index('sensing_time')).reset_index()

with filesystem.openbin('metadata/deimos_pan_metadata.pq', 'wb') as f:
    pan_median.to_parquet(f)